## Runnable Interface 介绍与使用

为了尽可能简化创建自定义链的过程，Langchain 实现了一个 **[Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)** 协议。

许多 LangChain 组件都实现了 Runnable 协议，包括 chat models, LLMs, output parsers, retrievers, prompt templates等等。此外，还有几个用于处理可运行对象的[有用原语](https://python.langchain.com/v0.1/docs/expression_language/primitives/)。

Runnable 是一个标准接口，包括：

- stream：流式返回生成内容（chunk）
- invoke：对输入调用该链
- batch：对输入列表调用该链

不同组件的输入和输出类型有所差异:

| 组件    | 输入类型                                           | 输出类型           |
| ------------ | ----------------------------------------------------- | --------------------- |
| Prompt       | Dictionary                                            | PromptValue           |
| ChatModel    | Single string, list of chat messages or a PromptValue | ChatMessage           |
| LLM          | Single string, list of chat messages or a PromptValue | String                |
| OutputParser | The output of an LLM or ChatModel                     | Depends on the parser |
| Retriever    | Single string                                         | List of Documents     |
| Tool         | Single string or dictionary, depending on the tool    | Depends on the tool   |


所有 Runnable 对象都显式描述输入和输出 Schema，以检查输入和输出格式：

- input_schema：从Runnable的结构自动生成的输入Pydantic模型
- output_schema：从Runnable的结构自动生成的输出Pydantic模型 


### Input Schema

为了演示如何使用，下面我们创建一个超级简单的PromptTemplate + ChatModel链。

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("讲个关于 {topic} 的笑话吧")
chain = prompt | model

#### schema 方法

一个描述 Runnable 接受的输入的说明。这是根据任何 Runnable 结构动态生成的 Pydantic 模型。您可以调用 .schema() 来获取 `JSONSchema` 表示。

In [2]:
# 查看 Chain 的输入类型
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [3]:
# 查看 Prompt 的输入类型（Chain的输入从 Prompt 开始，因此输入类型一致）
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [4]:
# 查看 Model 的输入类型
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'strin

### Output Schema

输出类型仍然可以调用 .schema() 来获取其 `JSONSchema` 表示。

In [5]:
# 查看 Chain 的输出类型
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'name': {'title': 'Name', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}

### Stream

使用 .stream() 方法查看（同步）流式输出结果

In [7]:
for s in chain.stream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

当然可以！这是一个关于程序员的笑话：

有一天，一位程序员走进酒吧，点了一杯酒。

酒保问：“你要喝什么？”

程序员回答：“随便，反正我只是一段代码，能编译就行。”

酒保无奈地说：“好吧，那我给你加点注释！”

程序员惊讶地问：“为什么要加注释？”

酒保笑着说：“因为我想确保你明白你喝的是什么！”

希望这个笑话能让你开心！

### Invoke
使用 .invoke() 方法单次（同步）调用

In [8]:
chain.invoke({"topic": "程序员"})

AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进咖啡店，点了一杯咖啡。店员问他：“要不要加糖？”\n\n程序员回答：“不，我只想要代码的‘空值’。”\n\n店员一脸懵：“那是什么？”\n\n程序员摇摇头：“没关系，反正你也不会理解。”\n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 18, 'total_tokens': 108}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None})

### Batch
使用 .batch() 方法（同步）批量调用

In [9]:
chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进酒吧，点了两杯酒。他喝了一口，觉得味道不错，于是又点了两杯。接着，他又喝了一口，觉得还不错，于是又点了两杯。\n\n酒保看着他，问：“你为什么总是点两杯？”\n\n程序员回答：“因为我习惯了双重确认！”', response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 18, 'total_tokens': 110}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='当然可以！这是一个关于产品经理的笑话：\n\n有一天，一个产品经理在会议上说：“我们需要一个功能，可以让用户一键完成所有操作！”\n\n团队的开发人员问：“那这个功能要怎么实现呢？”\n\n产品经理想了想，然后自信地回答：“很简单，给他们一个按钮，写上‘一键完成’就行了！”\n\n开发人员无奈地说：“可是用户会期待这个按钮能做什么呢？”\n\n产品经理微笑着说：“那就让他们自己想象吧！”\n\n这个笑话反映了产品经理常常对用户需求的理解和开发人员之间的幽默冲突！希望你喜欢！', response_metadata={'token_usage': {'completion_tokens': 142, 'prompt_tokens': 16, 'total_tokens': 158}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='当然可以！这是一个关于测试经理的笑话：\n\n有一天，测试经理去参加一个聚会，大家都在讨论各自的工作。\n\n一个开发人员说：“我昨天写了一个新功能，真是太棒了！”\n\n测试经理笑着说：“你确定吗？你有没有考虑到可能的缺陷？”\n\n开发人员自信地回答：

In [10]:
messages = chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

In [12]:
# 使用 StrOutputParser 来处理 Batch 批量输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

for idx, m in enumerate(messages):
    print(f"笑话{idx}:\n")
    print(output_parser.invoke(m))
    print("\n")

笑话0:

当然可以！这是一个关于程序员的笑话：

有一天，一位程序员走进酒吧，点了一杯酒。酒保问他：“你要什么样的酒？”

程序员回答：“给我一个‘bug-free’的饮料。”

酒保愣了一下：“抱歉，我们没有那种饮料。”

程序员说：“那就给我一杯‘无错误’的水吧！”

酒保无奈地摇了摇头：“你知道吗？水也是有‘bug’的。”

程序员叹了口气：“好吧，那就给我一杯‘测试版’的水吧！”

希望这个笑话能让你开心！


笑话1:

当然可以！这是一个关于产品经理的笑话：

有一天，产品经理、开发者和设计师一起去爬山。爬到一半，突然下起了大雨。

设计师说：“我们应该找个地方避雨，设计一个更好的避雨方案。”

开发者说：“不，我们应该先看看天气API，确认一下这场雨会持续多久！”

产品经理则说：“我们先来个用户调研，问问大家对雨天爬山的看法！”

最后，大家都淋成了落汤鸡，只有产品经理在旁边拿着雨伞说：“我觉得我们的雨伞功能可以再优化一下！”

希望这个笑话让你开心！


笑话2:

当然可以！这里有一个关于测试经理的笑话：

有一天，一个测试经理走进咖啡店，点了一杯咖啡。服务员问他：“您想要加糖吗？”

测试经理回答：“请先给我一个测试用例，确保加糖的功能正常！”

服务员无奈地说：“好吧，那我先给你咖啡，然后再测试加糖。”

测试经理点头：“记得记录下每一步，万一出错我需要追溯！”

服务员说：“明白了，不过如果出错，我会把责任推给开发者！”

测试经理笑着说：“那我就把它归类为‘不合格’！”

希望这个笑话能让你开心！




## 异步操作

这些方法也有相应的异步方法，应与 `asyncio` 的 `await` 语法一起使用以进行并发操作：

- astream：异步地流式返回生成内容（chunk）
- ainvoke：异步地对输入调用该链
- abatch：异步地对输入列表调用该链
- astream_log: 在发生时会返回中间步骤，并且最终返回结果之外。
- astream_events: beta 流式传输事件，在 langchain-core 0.1.14 中引入


### Async Stream

In [13]:
async for s in chain.astream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

当然可以！这里有一个关于程序员的笑话：

有一天，一位程序员走进酒吧，点了一杯啤酒。喝完后，他发现自己没有带钱包。于是，他对 bartender 说：“我可以用代码支付吗？”

bartender 想了想说：“当然可以，只要你能给我一个没有 bug 的程序！”

程序员沉默了片刻，然后回答：“那我可能得喝水了。”

希望你喜欢这个笑话！

### Async Invoke

In [14]:
await chain.ainvoke({"topic": "程序员"})

AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进酒吧，点了一杯酒。酒保问他：“你想要什么饮料？”\n\n程序员回答：“我想要一杯‘调试’。”\n\n酒保疑惑地问：“调试？那是什么饮料？”\n\n程序员笑着说：“就是你把酒倒到杯子里，然后再把酒倒回去，直到我满意为止！”\n\n希望这个笑话能让你开心！', response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 18, 'total_tokens': 124}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None})

### Async Batch

In [15]:
await chain.abatch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='当然可以！这里有一个关于程序员的笑话：\n\n有一天，程序员的妻子对他说：“你能帮我修一下厨房的水龙头吗？”\n\n程序员回答：“当然可以，不过你先给我一个详细的需求文档。”\n\n妻子生气地说：“我只想让你修水龙头，不是开发一个新程序！”\n\n程序员笑着说：“没关系，等我修好后，我会写个用户手册给你！”\n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 18, 'total_tokens': 127}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='当然可以！这是一个关于产品经理的笑话：\n\n为什么产品经理总是带着铅笔和纸？\n\n因为他们随时都要准备好“划重点”！', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 16, 'total_tokens': 54}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='当然可以！这是一个关于测试经理的笑话：\n\n有一天，测试经理带着团队去参加一个技术大会。在会议上，讲师问大家：“你们觉得软件测试最重要的是什么？”\n\n测试经理自信地回答：“是发现bug！”\n\n讲师点头称赞：“很好！那你们是怎么发现bug的呢？”\n\n测试经理笑着说：“我们用一种特别的方法：把软件给开发人员用！”\n\n全场哄笑，开发人员则默默低下了头。', response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 18, 'total